<a href="https://colab.research.google.com/github/hoa92ng/Homework/blob/main/Welcome_To_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
test

In [ ]:
using Whisper;
using Whisper.net;
using NAudio.Wave;
using System;
using System.Threading.Tasks;
using System.Runtime.InteropServices;

// See https://aka.ms/new-console-template for more information
// #region Naudio

class Program{

    [DllImport("user32")]
    public static extern int SetCursorPos(int x, int y);

    static WaveInEvent _waveSource = new WaveInEvent();
    // static int _silence_count = 0;
    // static int _temp_count = 0;
    static bool _is_active = false;
    static WhisperProcessor processor =  null;
    static List<float> _list_float = new List<float>();
    static List<byte> _list_byte = new List<byte>();
    static DateTime _last_sound_detection = DateTime.Now;
    static TimeSpan _silence_threshold = TimeSpan.FromSeconds(5); // Adjust the silence duration threshold as needed
    static float _silence_level_threshold = 20; // Adjust the silence level threshold as needed
    static void waveSource_DataAvailable(object? sender, WaveInEventArgs e)
    {
        var _value = Math.Abs(BitConverter.ToInt16(e.Buffer, (e.BytesRecorded - 2)));
        if (_value > _silence_level_threshold)
        {
            _last_sound_detection = DateTime.Now;

            byte[] buffer = e.Buffer;
            for (int index = 0; index < e.BytesRecorded; index += 2)
            {
                short sample = (short)((buffer[index + 1] << 8) |
                                        buffer[index]);
                float sample32 = sample / 32768f;
                _list_float.Add(sample32);
            }
            _list_byte = _list_byte.Concat(buffer.ToList()).ToList();
        }
        else
        {
            if (DateTime.Now - _last_sound_detection > _silence_threshold && _is_active && _list_byte.Count() > 0)
            {
                Console.WriteLine("Silence detected.");
                // stuff here
                Audio_Recognition(_list_float, processor).GetAwaiter().GetResult();
                _list_byte.Clear();
                _list_float.Clear();
                // Take action on silence detection, e.g., stop recording, log, etc.
            }
        }
    }

    void setCursorPos(int, int x, int y){

    }

    static async Task Audio_Recognition(List<float> _list_float, WhisperProcessor? processor){
        await foreach(var result in processor.ProcessAsync(_list_float.ToArray()))
        {
            Console.WriteLine($"{result.Start}->{result.End}: {result.Text}");
        }
    }
    static void Main(string[] args){
        _waveSource.WaveFormat = new WaveFormat(16000, 1);
        var whisperFactory = WhisperFactory.FromPath(@"ggml-base.en.bin");
        processor = whisperFactory.CreateBuilder()
            .WithLanguage("auto")
            .Build();

        Console.WriteLine("Please select function: 1. Start Record    2. Stop Record    3. Stream    Other.Exit");
        while(true){
            switch(Console.ReadLine()){
                case "1":
                    _list_float.Clear();
                    _waveSource.StartRecording();
                    _waveSource.DataAvailable += waveSource_DataAvailable;
                    break;
                case "2":
                    _is_active = false;
                    _waveSource.StopRecording();
                    _waveSource.DataAvailable -= waveSource_DataAvailable;
                    // run
                    Audio_Recognition(_list_float, processor).GetAwaiter().GetResult();
                    break;
                case "3":
                    _list_float.Clear();
                    _waveSource.StartRecording();
                    _waveSource.DataAvailable += waveSource_DataAvailable;
                    _is_active = true;
                    break;
                default:
                    _waveSource.Dispose();
                    return;
            }
            Console.WriteLine("Please select function: 1. Start Record    2. Stop Record    Other.Exit");
        }
    }
}



